## Part 1 - Retrieve the Data from the API

Now that we understand how the API request is constructed, let’s start working with our Python script. First, we need to load all of the necessary libraries.


In [2]:
import os
import pandas as pd
import requests

%matplotlib inline
from matplotlib import pyplot as plt

To communicate with the API and dynamically request data for different combinations of state FIPS codes, early years, and late years, we need to employ the Requests library. Requests is a library designed to allow users to send HTTP requests with Python. In the function below, we use the method .get to request data from API and load it into a Response object r. The Requests library includes a built in JSON decoder to read the JSON data that is returned and load it into a Pandas dataframe.

In [3]:
def create_dataframe(fips,early_year,late_year):
    """
    Submits request for data to the API and creates formatted pandas dataframe for use with clustering algorithms
    
    fips = string defining the 2-digit FIPS code for your state you of interest
    early_year = string defining the 1st date of change period (1996, 2001, 2006 or 2010)
    late_year = string defining the 2nd date of change period (1996, 2001, 2006 or 2010)
    """
    # Format inputs for url
    fips = "'{}'".format(fips)
    early_year = "'{}'".format(early_year)
    late_year = "'{}'".format(late_year)
    
    #API request for Land Cover data in JSON format
    url = "https://coast.noaa.gov/opendata/LandCover/api/v1/distributionOfChangeGainsLossesByLandcovers?$filter=startswith(geoId, {0}) and length(geoId) eq 5 and earlyYear eq {1} and lateYear eq {2}".format(fips, early_year, late_year)
    
    r = requests.get(url, headers={'Accept': 'application/json'})
    data = r.json()
    lca_df = pd.DataFrame(data['value'])
    
    # Set index to the FIPS code and drop Early and Late Year Fields
    lca_df_index = lca_df.set_index('geoId').drop(['earlyYear', 'lateYear'], axis=1)
    
    return lca_df_index

Now we will execute the function. The C-CAP Atlas has county level land cover change statistics organized by FIPS code and dates of change. C-CAP has mapped land cover for the years 1996, 2001, 2006 and 2010 using imagery acquired by the Landsat suite of satellites. This means we can look at changes in land cover that have occurred over multiple time series such as coarse intervals like 1996-2010 or we can look at finer scale changes over a 5 year period such as 2001-2006.  For this example, again we will select the state of Connecticut (FIPS code:9) and request land cover changes occurring from 2001-2010. 

In [4]:
CT_2001_2010 = create_dataframe('09',2001,2010)

Let’s confirm our new object contains the correct data using the method .head( ) 

In [5]:
CT_2001_2010.head()

,AgrAreaGain,AgrAreaLoss,BarAreaGain,BarAreaLoss,EmwAreaGain,EmwAreaLoss,ForAreaGain,ForAreaLoss,GrsAreaGain,GrsAreaLoss,...,LIDAreaGain,LIDAreaLoss,OSDAreaGain,OSDAreaLoss,SscbAreaGain,SscbAreaLoss,WdwAreaGain,WdwAreaLoss,WtrAreaGain,WtrAreaLoss
geoId,,,,,,,,,,,,,,,,,,,,,
09001,0.31,-1.64,0.43,-0.20,0.06,-0.16,0.67,-7.58,0.41,-0.30,...,4.33,-0.49,3.61,-1.77,0.49,-0.86,0.05,-0.36,0.05,-0.20
09003,0.71,-6.39,0.48,-0.19,0.14,-0.19,0.23,-10.56,1.06,-0.17,...,6.25,-0.45,6.91,-1.83,0.95,-1.95,0.08,-0.81,0.06,-0.26
09005,0.60,-1.03,0.31,-0.06,0.29,-0.10,0.20,-3.89,1.36,-0.22,...,0.77,-0.06,0.97,-0.20,0.84,-0.36,0.13,-0.22,0.12,-0.28
09007,0.30,-0.92,0.30,-0.09,0.12,-0.09,0.11,-4.09,0.24,-0.16,...,1.72,-0.11,1.37,-0.37,0.69,-0.47,0.05,-0.32,0.08,-0.15
09009,0.36,-1.92,0.49,-0.27,0.38,-0.13,0.20,-8.45,0.75,-0.59,...,4.05,-0.43,3.93,-1.55,0.71,-1.01,0.06,-0.42,0.15,-0.48


We can also get a list of the column names by using the following code

In [6]:
list(CT_2001_2010.columns.values)

['AgrAreaGain',
 'AgrAreaLoss',
 'BarAreaGain',
 'BarAreaLoss',
 'EmwAreaGain',
 'EmwAreaLoss',
 'ForAreaGain',
 'ForAreaLoss',
 'GrsAreaGain',
 'GrsAreaLoss',
 'HIDAreaGain',
 'HIDAreaLoss',
 'LIDAreaGain',
 'LIDAreaLoss',
 'OSDAreaGain',
 'OSDAreaLoss',
 'SscbAreaGain',
 'SscbAreaLoss',
 'WdwAreaGain',
 'WdwAreaLoss',
 'WtrAreaGain',
 'WtrAreaLoss']